Pytorch 튜토리얼
- https://tutorials.pytorch.kr/beginner/basics/quickstart_tutorial.html

Pytorch의 데이터 작업을 위한 기본요소 두가지
1. torch.utils.data.DataLoader : Dataset을 순회 가능한 객체(iterable)로 감싼다. 
2. torch.utils.data.Dataset : 샘플과 정답(label)저장

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

#  TorchText, TorchVision ,TorchAudio 의 도메인 특화 라이브러리를 데이터셋과 함께 제공
from torchvision import datasets
from torchvision.transforms import ToTensor

데이터 셋 다운로드
- FashionMNIST 데이터셋 : 셔츠, 운동화, 샌들 등 작은 이미지들의 모음, 28*28픽셀 70,000개로 구성

In [2]:
# 공개 데이터셋에서 학습 데이터를 내려받습니다.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# 공개 데이터셋에서 테스트 데이터를 내려받습니다.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
display(training_data,test_data)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

Dataset을 DataLoader의 인자로 전달
- 데이터셋을 iterable객체로 변환
    - 자동화 batch, sampling, shuffle, multiprocess data loading 지원

In [4]:
batch_size = 64
# 배치크기 64로 정의
# 데이터로더 객체의 각 요소는 64개의 feature와 label을 묶음(batch)로 반환

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


모델만들기
- 신경망 모델은 nn.Module을 상속받는 class 생성하여 정의
- __init__ 함수에서 신경망 layer를 정의
- forward 함수에서 신경망에 데이터를 어떻게 전달할지 지정
- 가능한 경우 GPU, MPS로 신경망을 이동, 연산을 가속화

In [5]:
# device 확인
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [6]:
# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # 입력데이터 평탄화
                                    # ex) 이미지데이터의 경우 2차원 픽셀값의 행렬형태
                                    #     -> 이러한 2차원 구조를 1차원 구조로 변환
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # 첫번째 레이어 정의(입력 28*28 / 출력 512)
            nn.ReLU(), # 활성화 함수
            nn.Linear(512, 512), # 두번째 레이어 정의
            nn.ReLU(),
            nn.Linear(512, 10) # 최종 레이어 정의 (10개의 출력을 갖는다.)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x) # 앞서 정의한 선형레이어들을 통해 전파
        return logits

model = NeuralNetwork().to(device) # NeuralNetwork의 인스턴스를 생성, device로 이동
print(model) # 구조 출력

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델 매개변수 최적화
- 모델 학습 시 손실함수, 옵티마이저가 필요

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

각 training loop에서 배치모델은 학습 데이터셋에 대한 예측 수행, 예측 오류를 역전파하여 모델의 매개변수 조정.

In [10]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X) # 모델엥 입력데이터 전달, 예측수행
        loss = loss_fn(pred, y) # 예측값, 실제값 간의 손실 계산

        # 역전파
        optimizer.zero_grad() # 이전에 계산된 기울기 초기화
        loss.backward() # 손실에 대한 역전파 수행
        optimizer.step() # optimizer 사용, 모델 매개변수 업데이트

        if batch % 100 == 0: # 100번의 배치마다 현재 손실, 진행상태 출력
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

모델이 학습하고 있는지 확인하기 위해 테스트 데이터셋으로 모델 성능 확인

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # 모델을 평가모드로 설정, 드롭아웃과 같은 계층이 활성화되지 않도록..
    test_loss, correct = 0, 0 # 테스트 손실, 정확한 예측 수 초기화
    with torch.no_grad(): # 테스트중에는 기울기를 계산하지 않음
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X) # 모델에 입력데이터를 전달, 예측 수행

            # 값 누적
            test_loss += loss_fn(pred, y).item() 
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches # 평균손실 계산
    correct /= size # 정확도 계산
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

학습단계는 여러번의 epochs(반복단계)를 거쳐서 수행. 각 epoch 에서 모델은 더 나은 예측을 하기위해 매개변수를 학습. 각 epoch마다 모델의 정확도, 손실을 출력
- 아래를 통해 epoch마다 정확도, 손실의 증감을 확인

In [12]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.319786  [   64/60000]
loss: 2.301068  [ 6464/60000]
loss: 2.283386  [12864/60000]
loss: 2.272886  [19264/60000]
loss: 2.252284  [25664/60000]
loss: 2.225527  [32064/60000]
loss: 2.230992  [38464/60000]
loss: 2.197194  [44864/60000]
loss: 2.196517  [51264/60000]
loss: 2.159256  [57664/60000]
Test Error: 
 Accuracy: 45.1%, Avg loss: 2.158753 

Epoch 2
-------------------------------
loss: 2.173347  [   64/60000]
loss: 2.159907  [ 6464/60000]
loss: 2.101860  [12864/60000]
loss: 2.119093  [19264/60000]
loss: 2.059775  [25664/60000]
loss: 2.004384  [32064/60000]
loss: 2.032196  [38464/60000]
loss: 1.952038  [44864/60000]
loss: 1.957828  [51264/60000]
loss: 1.882748  [57664/60000]
Test Error: 
 Accuracy: 52.8%, Avg loss: 1.887966 

Epoch 3
-------------------------------
loss: 1.918993  [   64/60000]
loss: 1.885367  [ 6464/60000]
loss: 1.773024  [12864/60000]
loss: 1.821536  [19264/60000]
loss: 1.699727  [25664/60000]
loss: 1.661464  [32064/600

모델 저장

In [13]:
torch.save(model.state_dict(), "model.pth")

모델 불러오기

In [14]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [15]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1] # test_data에서 첫 번째 샘플의 입력, 레이블을 가져옴
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y] 
    # armax로 예측된 클래스의 인덱스를 구함
    # 해당 인덱스에 해당하는 클래스 이름을 classes에서 가져와 예측값을 출력
 
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
